In [67]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [68]:
# #import sklearn
# from sklearn.cross_validation import train_test_split
# from sklearn import metrics
# from sklearn.cross_validation import cross_val_score
# from sklearn.linear_model import LogisticRegression
#from sklearn import feature_selection as fs #maybe use for p-vals...

In [69]:
# import patsy #not sure if we'll need this

In [70]:
# read the data in
df = pd.read_csv("http://www.ats.ucla.edu/stat/data/binary.csv")

In [71]:
df.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [72]:
df.count()

admit    400
gre      400
gpa      400
rank     400
dtype: int64

In [73]:
df.columns[3]

'rank'

In [74]:
df.columns=['admit','gre','gpa','prestige']

In [75]:
df.head()

,admit,gre,gpa,prestige
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [76]:
dummy_ranks = pd.get_dummies(df['prestige'], prefix='prestige') #pd.get_dummies converts a specified categorical variabe into a series of dummy vars
dummy_ranks.head()

,prestige_1,prestige_2,prestige_3,prestige_4
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


In [77]:
#pd.value_counts(df['prestige'])

In [78]:
data = df[['admit','gre','gpa']].join(dummy_ranks.ix[:,'prestige_2':])

In [79]:
data.head()

,admit,gre,gpa,prestige_2,prestige_3,prestige_4
0,0,380,3.61,0,1,0
1,1,660,3.67,0,1,0
2,1,800,4.00,0,0,0
3,1,640,3.19,0,0,1
4,0,520,2.93,0,0,1


In [80]:
train_cols = data.columns[1:]
train_cols

Index(['gre', 'gpa', 'prestige_2', 'prestige_3', 'prestige_4'], dtype='object')

In [81]:
#logit = sm.Logit(data['admit'],data[train_cols]) #How to run a logit with statsmodels.Logit()

In [82]:
#result = logit.fit()

In [83]:
#summary = result.summary()

In [84]:
#summary

In [85]:
#type(summary)

In [86]:
# This is the sklearn way:
# model = LogisticRegression()
# model = model.fit(data[train_cols],data['admit'])
# model.score(data[train_cols],data['admit'])

In [87]:
# data['admit'].mean()

In [88]:
# type(model)

In [89]:
# model

In [90]:
# model.coef_

In [91]:
# train_cols

In [92]:
#Use statsmodels over scikit learn for p-vals
logit = sm.Logit(data['admit'],data[train_cols]) #How to run a logit with statsmodels.Logit()
result = logit.fit()
summary = result.summary()
summary

Optimization terminated successfully.
         Current function value: 0.589306
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  400
Model:                          Logit   Df Residuals:                      395
Method:                           MLE   Df Model:                            4
Date:                Fri, 01 Apr 2016   Pseudo R-squ.:                 0.05707
Time:                        11:36:36   Log-Likelihood:                -235.72
converged:                       True   LL-Null:                       -249.99
                                        LLR p-value:                 9.732e-06
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0014      0.001      1.329      0.184        -0.001     0.003
gpa           -0.1336      0.194     -0.687      0.492        -0.515     0.248
prestige_2    -0.9663      0.300     -3.216      0.001        -1.555    -0.377
prestige_3    -1.5451      0.332     -4.648      0.000        -2.197    -0.894
prestige_4    -1.8776      0.402     -4.676      0.000        -2.665    -1.091
==============================================================================
"""

In [93]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400 entries, 0 to 399
Data columns (total 6 columns):
admit         400 non-null int64
gre           400 non-null int64
gpa           400 non-null float64
prestige_2    400 non-null float64
prestige_3    400 non-null float64
prestige_4    400 non-null float64
dtypes: float64(4), int64(2)
memory usage: 21.9 KB


In [94]:
pd.DataFrame(result.pvalues)

,0
gre,0.183932
gpa,0.492152
prestige_2,0.001298
prestige_3,0.000003
prestige_4,0.000003


In [107]:
numConditions=5
conditionCodes = list(range(numConditions))
conditionCodes

[0, 1, 2, 3, 4]

In [108]:
startCode = np.random.choice(conditionCodes)
startCode

2

In [109]:
assignList = conditionCodes[startCode:] + conditionCodes[:startCode]

In [110]:
assignList = ((len(data)//numConditions)*assignList) + assignList[:(len(data)%numConditions)]

In [111]:
len(assignList)

400

In [112]:
data['condition'] = assignList

In [113]:
#Now try multinomial logit of condition on gre, gpa, prestige_2, prestige_3, prestige_4 
data.head()

,admit,gre,gpa,prestige_2,prestige_3,prestige_4,condition
0,0,380,3.61,0,1,0,2
1,1,660,3.67,0,1,0,3
2,1,800,4.00,0,0,0,4
3,1,640,3.19,0,0,1,0
4,0,520,2.93,0,0,1,1


In [114]:
# # Run logit of treat 1 on control - compare to results from mlogit test
# data_treat1 = data[data['condition']!=2]
# data_treat1.describe()

In [115]:
# logit_treat1 = sm.Logit(data_treat1['condition'],data_treat1[train_cols]) #How to run a logit with statsmodels.Logit()
# result_treat1 = logit_treat1.fit()
# summary_treat1 = result_treat1.summary()
# summary_treat1

In [116]:
# Multinomial Logit with statsmodels -- use when there are more than 2 conditions; works for logit, too :)
mlogit = sm.MNLogit(data['condition'],data[train_cols])
mresult = mlogit.fit()
msummary = mresult.summary()
msummary

Optimization terminated successfully.
         Current function value: 1.602087
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:              condition   No. Observations:                  400
Model:                        MNLogit   Df Residuals:                      380
Method:                           MLE   Df Model:                           16
Date:                Fri, 01 Apr 2016   Pseudo R-squ.:                0.004568
Time:                        11:37:52   Log-Likelihood:                -640.83
converged:                       True   LL-Null:                       -643.78
                                        LLR p-value:                    0.9893
===============================================================================
condition=1       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
gre            -0.0007      0.001     -0.449      0.654        -0.004     0.002
gpa             0.1393      0.277      0.502      0.616        -0.404     0.683
prestige_2     -0.2455      0.451     -0.544      0.586        -1.129     0.639
prestige_3      0.1057      0.473      0.224      0.823        -0.821     1.033
prestige_4     -0.1666      0.540     -0.308      0.758        -1.225     0.892
-------------------------------------------------------------------------------
condition=2       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
gre             0.0011      0.001      0.732      0.464        -0.002     0.004
gpa            -0.2779      0.282     -0.984      0.325        -0.832     0.276
prestige_2      0.2540      0.474      0.536      0.592        -0.675     1.183
prestige_3      0.5710      0.500      1.141      0.254        -0.409     1.551
prestige_4      0.3001      0.557      0.538      0.590        -0.792     1.393
-------------------------------------------------------------------------------
condition=3       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
gre             0.0002      0.001      0.168      0.866        -0.003     0.003
gpa            -0.0400      0.279     -0.143      0.886        -0.587     0.507
prestige_2     -0.0457      0.450     -0.102      0.919        -0.927     0.836
prestige_3     -0.0059      0.486     -0.012      0.990        -0.958     0.947
prestige_4     -0.0095      0.537     -0.018      0.986        -1.062     1.043
-------------------------------------------------------------------------------
condition=4       coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
gre            -0.0008      0.001     -0.526      0.599        -0.004     0.002
gpa             0.1288      0.277      0.466      0.641        -0.413     0.671
prestige_2     -0.0399      0.454     -0.088      0.930        -0.930     0.850
prestige_3     -0.0005      0.489     -0.001      0.999        -0.958     0.957
prestige_4      0.2179      0.526      0.414      0.679        -0.814     1.250
===============================================================================
"""

In [133]:
pVals = mresult.pvalues
pVals
#Columns are labeled by index which is confusion -- correspond to the (column# + 1) condition

,0,1,2,3
gre,0.653563,0.464278,0.866205,0.598899
gpa,0.615592,0.325172,0.885948,0.641335
prestige_2,0.586249,0.592151,0.919057,0.930077
prestige_3,0.823133,0.253663,0.990348,0.999163
prestige_4,0.757721,0.590323,0.985899,0.678895


In [134]:
pVals.columns = conditionCodes[1:] #Rename columns to correspond to conditions
pVals=pVals.reset_index()
pVals.rename(columns={'index':'variable'},inplace=True)

In [135]:
pVals

,variable,1,2,3,4
0,gre,0.653563,0.464278,0.866205,0.598899
1,gpa,0.615592,0.325172,0.885948,0.641335
2,prestige_2,0.586249,0.592151,0.919057,0.930077
3,prestige_3,0.823133,0.253663,0.990348,0.999163
4,prestige_4,0.757721,0.590323,0.985899,0.678895


In [137]:
#Sample syntax
for index, row in pVals.iterrows():
     print(row[1])

0.6535625364300499
0.6155917457024278
0.5862490265123474
0.8231334297028365
0.7577208148632719


In [144]:
#Next: check in any values are <= .05; if they are - randomize again!